In [6]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
from fpdf import FPDF
import tkinter.filedialog as fd
import csv
import os

class BookStoreApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Sistem Pembelian Buku")
        
        screen_width = root.winfo_screenwidth()
        screen_height = root.winfo_screenheight()
        self.root.geometry(f"{screen_width}x{screen_height}")

        self.user_file = 'user_data.csv'
        self.user_details_file = 'user_details.csv'
        self.initialize_csv_files()

        self.cart = []
        self.login_page()

    def initialize_csv_files(self):
        if not os.path.exists(self.user_file):
            with open(self.user_file, 'w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(['username', 'password'])

        if not os.path.exists(self.user_details_file):
            with open(self.user_details_file, 'w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(['username', 'full_name', 'phone_number', 'address'])

    def login_page(self):
        self.clear_frame()
        self.frame = tk.Frame(self.root, bg='White')
        self.frame.pack(fill='both', expand=True)

        screen_width = self.root.winfo_screenwidth()
        screen_height = self.root.winfo_screenheight()

        img_path = "Halaman Login 1.png"
        if os.path.exists(img_path):
            login_image = Image.open(img_path)
            login_image = login_image.resize((screen_width, screen_height))
            login_photo = ImageTk.PhotoImage(login_image)
            login_label = tk.Label(self.frame, image=login_photo, bg='White')
            login_label.image = login_photo
            login_label.place(x=0, y=0, relwidth=1, relheight=1)  

        tk.Label(text='Selamat Datang', font=('Arial', 38, 'bold') ,bg='#235ba8', fg='white').place(x=870, y=245)
        tk.Label(text='Junior Book', font=('Courier New', 29, 'bold') ,bg='#235ba8', fg='white').place(x=930, y=310)
        
        self.username_entry = tk.Entry(width=25, font=('Microsoft Yahei UI Light', 11, 'bold'), bg='#235ba8', bd=0, fg='white', highlightthickness=0)
        self.username_entry.place(x=950, y=370)
        self.username_entry.insert(0, 'Username')

        def clear_username_entry(event):
            if self.username_entry.get() == 'Username':
                self.username_entry.delete(0, tk.END)

        self.username_entry.bind("<FocusIn>", clear_username_entry)
        self.username_entry.bind("<Return>", self.focus_password_entry)

        frame1 = tk.Frame(width=250, height=2, bg='white')
        frame1.place(x=950, y=390)

        self.password_entry = tk.Entry(width=25, font=('Microsoft Yahei UI Light', 11, 'bold'), bg='#235ba8', bd=0, fg='white', highlightthickness=0)
        self.password_entry.place(x=950, y=420)
        self.password_entry.insert(0, 'Password')

        def clear_password_entry(event):
            if self.password_entry.get()=='Password': 
                self.password_entry.delete(0, tk.END)

        self.password_entry.bind("<FocusIn>", clear_password_entry)
        self.password_entry.bind("<Return>", self.authenticate)

        frame2 = tk.Frame(width=250, height=2, bg='white')
        frame2.place(x=950, y=440)

        def hide():
            openeye.config(file='closeye.png')
            self.password_entry.config(show='*')
            eyeButton.config(command=show)

        def show():
            openeye.config(file='openeye.png')
            self.password_entry.config(show='')
            eyeButton.config(command=hide)

        openeye=tk.PhotoImage(file='openeye.png')
        eyeButton=tk.Button(image=openeye, bd=0, bg='#235ba8', activebackground='#235ba8', cursor='hand2', command=hide)
        eyeButton.place(x=1170, y=410) 

        loginButton = tk.Button(text='Login', font=('Open Sans', 16, 'bold'),
                                fg='#235ba8', bg='#fed100', activeforeground='#235ba8',
                                activebackground='#fed100', cursor='hand2', bd=0, width=19, highlightthickness=0, command=self.authenticate)
        loginButton.place(x=950, y=470)
        
        signupLabel = tk.Label(text='Belum Punya Akun?', font=('Open Sans', 10, 'bold'), fg='white', bg='#235ba8')
        signupLabel.place(x=950, y=515)

        newaccountButton = tk.Button(text='Daftar', font=('Open Sans', 10, 'bold'),
                                     fg='#235ba8', bg='#fed100', activeforeground='#235ba8',
                                     activebackground='#fed100', cursor='hand2', bd=0, width=2, highlightthickness=0, command=self.register_page)
        newaccountButton.place(x=1070, y=520)

    def focus_password_entry(self, event):
        self.password_entry.focus()

    def register_page(self):
        self.clear_frame()
        self.frame = tk.Frame(self.root, bg='White')
        self.frame.pack(fill='both', expand=True)

        screen_width = self.root.winfo_screenwidth()
        screen_height = self.root.winfo_screenheight()

        img_path = "Halaman Login 1.png"
        if os.path.exists(img_path):
            register_image = Image.open(img_path)
            register_image = register_image.resize((screen_width, screen_height))
            register_photo = ImageTk.PhotoImage(register_image)
            register_label = tk.Label(self.frame, image=register_photo, bg='White')
            register_label.image = register_photo
            register_label.place(x=0, y=0, relwidth=1, relheight=1)  

        tk.Label(text='Halaman Registrasi Akun', font=('Arial', 38, 'bold') ,bg='#235ba8', fg='white').place(x=870, y=260)
        tk.Label(text='Junior Book', font=('Courier New', 29, 'bold') ,bg='#235ba8', fg='white').place(x=930, y=310)
        
        self.reg_username_entry = tk.Entry(width=25, font=('Microsoft Yahei UI Light', 11, 'bold'), bg='#235ba8', bd=0, fg='white', highlightthickness=0)
        self.reg_username_entry.place(x=950, y=370)
        self.reg_username_entry.insert(0,'Username')

        def clear_reg_username_entry(event):
            if self.reg_username_entry.get() == 'Username':
                self.reg_username_entry.delete(0, tk.END)

        self.reg_username_entry.bind("<FocusIn>", clear_reg_username_entry)

        frame1 = tk.Frame(width=250, height=2, bg='white')
        frame1.place(x=950, y=390)

        self.reg_password_entry = tk.Entry(width=25, font=('Microsoft Yahei UI Light', 11, 'bold'), bg='#235ba8', bd=0, fg='white', highlightthickness=0)
        self.reg_password_entry.place(x=950, y=420)
        self.reg_password_entry.insert(0, 'Password')

        def reg_clear_password_entry(event):
            if self.reg_password_entry.get()=='Password': 
                self.reg_password_entry.delete(0, tk.END)

        self.reg_password_entry.bind("<FocusIn>", reg_clear_password_entry)
        self.reg_password_entry.bind("<Return>", self.additional_info_page)

        frame2 = tk.Frame(width=250, height=2, bg='white')
        frame2.place(x=950, y=440)

        def hide():
            openeye.config(file='closeye.png')
            self.reg_password_entry.config(show='*')
            eyeButton.config(command=show)

        def show():
            openeye.config(file='openeye.png')
            self.reg_password_entry.config(show='')
            eyeButton.config(command=hide)

        openeye=tk.PhotoImage(file='openeye.png')
        eyeButton=tk.Button(image=openeye, bd=0, bg='#235ba8', activebackground='#235ba8', cursor='hand2', command=hide)
        eyeButton.place(x=1170, y=410) 

        registerButton = tk.Button(text='Registrasi', font=('Open Sans', 16, 'bold'),
                                fg='#235ba8', bg='#fed100', activeforeground='#235ba8',
                                activebackground='#fed100', cursor='hand2', bd=0, width=19, highlightthickness=0, command=self.additional_info_page)
        registerButton.place(x=950, y=470)
        backButton = tk.Button(text='Kembali', font=('Open Sans', 16, 'bold'),
                                     fg='#235ba8', bg='#fed100', activeforeground='#235ba8',
                                     activebackground='#fed100', cursor='hand2', bd=0, width=19, highlightthickness=0, command=self.login_page)
        backButton.place(x=950, y=510)

    def focus_reg_password_entry(self, event):
        self.reg_password_entry.focus()

    def additional_info_page(self, event=None):
        username = self.reg_username_entry.get()
        password = self.reg_password_entry.get()

        if username and password:
            self.clear_frame()
            self.frame = tk.Frame(self.root, bg='White')
            self.frame.pack(fill='both', expand=True)
        
            screen_width = self.root.winfo_screenwidth()
            screen_height = self.root.winfo_screenheight()

            img_path = "Halaman Login 1.png"
            if os.path.exists(img_path):
                tambahan_image = Image.open(img_path)
                tambahan_image = tambahan_image.resize((screen_width, screen_height))
                tambahan_photo = ImageTk.PhotoImage(tambahan_image)
                tambahan_label = tk.Label(self.frame, image=tambahan_photo, bg='White')
                tambahan_label.image = tambahan_photo
                tambahan_label.place(x=0, y=0, relwidth=1, relheight=1)  

            tk.Label(text='Informasi Tambahan', font=('Arial', 40, 'bold') ,bg='#235ba8', fg='white').place(x=870, y=260)
            
            self.full_name_entry = tk.Entry(width=25, font=('Microsoft Yahei UI Light', 11, 'bold'), bg='#235ba8', bd=0, fg='white', highlightthickness=0)
            self.full_name_entry.place(x=950, y=370)
            self.full_name_entry.insert(0, 'Nama Lengkap')

            def clear_full_name_entry(event):
                if self.full_name_entry.get() == 'Nama Lengkap':
                    self.full_name_entry.delete(0, tk.END)

            self.full_name_entry.bind("<FocusIn>", clear_full_name_entry)

            frame1 = tk.Frame(width=250, height=2, bg='white')
            frame1.place(x=950, y=390)

            self.phone_entry = tk.Entry(width=25, font=('Microsoft Yahei UI Light', 11, 'bold'), bg='#235ba8', bd=0, fg='white', highlightthickness=0)
            self.phone_entry.place(x=950, y=420)
            self.phone_entry.insert(0, 'No. Handphone')

            def clear_phone_entry(event):
                if self.phone_entry.get()=='No. Handphone': 
                    self.phone_entry.delete(0, tk.END)

            self.phone_entry.bind("<FocusIn>", clear_phone_entry)

            frame2 = tk.Frame(width=250, height=2, bg='white')
            frame2.place(x=950, y=440)

            self.address_entry = tk.Entry(width=25, font=('Microsoft Yahei UI Light', 11, 'bold'), bg='#235ba8', bd=0, fg='white', highlightthickness=0)
            self.address_entry.place(x=950, y=470)
            self.address_entry.insert(0, 'Alamat Lengkap')

            def clear_address_entry(event):
                if self.address_entry.get()=='Alamat Lengkap': 
                    self.address_entry.delete(0, tk.END)

            self.address_entry.bind("<FocusIn>", clear_address_entry)

            frame3 = tk.Frame(width=250, height=2, bg='white')
            frame3.place(x=950, y=490)

            tk.Button(text="Simpan",font=('Open Sans', 16, 'bold'),fg='#235ba8', bg='#fed100', activeforeground='#235ba8',
                                    activebackground='#fed100', cursor='hand2', bd=0, width=19, highlightthickness=0,
                                    command=lambda: self.save_registration(username, password)).place(x=950, y=540)
            tk.Button(text="Kembali", font=('Open Sans', 16, 'bold'),fg='#235ba8', bg='#fed100', activeforeground='#235ba8',
                                    activebackground='#fed100', cursor='hand2', bd=0, width=19, highlightthickness=0,
                                    command=self.login_page).place(x=950, y=570)
        else:
            messagebox.showerror("Registrasi", "Username dan Password tidak boleh kosong!")

    def save_registration(self, username, password, event=None):
        full_name = self.full_name_entry.get()
        phone_number = self.phone_entry.get()
        address = self.address_entry.get()

        if username and password and full_name and phone_number and address:
            if not self.user_exists(username):
                with open(self.user_file, 'a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow([username, password])
                with open(self.user_details_file, 'a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow([username, full_name, phone_number, address])
                messagebox.showinfo("Registrasi", "Registrasi berhasil!")
                self.login_page()
            else:
                messagebox.showerror("Registrasi", "Username sudah terdaftar!")
        else:
            messagebox.showerror("Registrasi", "Semua kolom harus diisi!")

    def user_exists(self, username):
        with open(self.user_file, 'r') as file:
            reader = csv.reader(file)
            next(reader)
            for row in reader:
                if row[0] == username:
                    return True
        return False

    def authenticate(self, event=None):
        username = self.username_entry.get()
        password = self.password_entry.get()
        
        if self.check_credentials(username, password):
            self.main_menu()
        else:
            messagebox.showerror("Login", "Username atau Password salah atau belum terdaftar!")

    def check_credentials(self, username, password):
        with open(self.user_file, 'r') as file:
            reader = csv.reader(file)
            next(reader)
            for row in reader:
                if row[0] == username and row[1] == password:
                    return True
        return False

    def main_menu(self):
        self.clear_frame()
        self.frame = tk.Frame(self.root, bg='White')
        self.frame.pack(fill='both', expand=True)

        tk.Label(self.frame, text="Menu Utama").pack(pady=10)
        tk.Button(self.frame, text="Pilih Produk", command=self.select_product).pack(pady=10)
        tk.Button(self.frame, text="Keranjang", command=self.view_cart).pack(pady=10)
        tk.Button(self.frame, text="Keluar", command=self.root.destroy).pack(pady=10)

    def select_product(self):
        self.clear_frame()
        self.frame = tk.Frame(self.root, bg='White')
        self.frame.pack(fill='both', expand=True)

        tk.Label(self.frame, text="Pilih Produk").grid(row=0, column=0, columnspan=5, pady=10)

        products = [
            {"name": "100 Kata Pertamaku", "image": "FOTO PRODUK PRAKTIKUM/100 Kata Pertamaku/100_kata_pertamaku-1.jpg", "price": "Rp89,000"},
            {"name": "Aktivitas Pertamaku ABC 123", "image": "FOTO PRODUK PRAKTIKUM/Aktivitas Pertamaku ABC/ABC-1.jpg", "price": "Rp89,000"},
            {"name": "100 Hewan Pertamaku", "image": "FOTO PRODUK PRAKTIKUM/100 Hewan Pertamaku/100_hewan-1.jpg", "price": "Rp89,000"},
            {"name": "Aku Mau Tau Semua - Jilid 1", "image": "FOTO PRODUK PRAKTIKUM/Aku Mau Tau Semua - Jilid 1/amts_v1-1.jpg", "price": "Rp59,000"},
            {"name": "Aku Mau Tau Semua - Jilid 2", "image": "FOTO PRODUK PRAKTIKUM/Aku Mau Tau Semua - Jilid 2/amts_v2-1.jpg", "price": "Rp59,000"},
            {"name": "Eduscience Eksperimen Keren", "image": "FOTO PRODUK PRAKTIKUM/Eduscience Eksperimen Keren/edu_eksperimen_keren-1.jpg", "price": "Rp64,000"},
            {"name": "Eduscience Hewan Predator", "image": "FOTO PRODUK PRAKTIKUM/Eduscience Hewan Predator/edu_hewan_predator-1.jpg", "price": "Rp64,000"},
            {"name": "Flip & Find - Lets See Inside Animal!", "image": "FOTO PRODUK PRAKTIKUM/Flip & Find - Lets See Inside Animal!/lets_see-1.jpg", "price": "Rp99,000"},
            {"name": "Kidzopedia Hewan Dan Tumbuhan", "image": "FOTO PRODUK PRAKTIKUM/Kidzopedia Hewan Dan Tumbuhan/kidzo_hewan_tumbuhan-1.jpg", "price": "Rp89,000"},
            {"name": "Kidzopedia Keajaiban Tubuh Manusia", "image": "FOTO PRODUK PRAKTIKUM/Kidzopedia Keajaiban Tubuh Manusia/kidzo_tubuh-1.jpg", "price": "Rp89,000"},
            {"name": "Kidzopedia Peristiwa Alam", "image": "FOTO PRODUK PRAKTIKUM/Kidzopedia Peristiwa Alam/kidzo_peristiwa_alam-1.jpg", "price": "Rp89,000"},
            {"name": "Lets Explore! Around the Farm", "image": "FOTO PRODUK PRAKTIKUM/Lets Explore! Around the Farm/farm-1.jpg", "price": "Rp98,000"},
            {"name": "Little Star 15 menit Pintar Membaca", "image": "FOTO PRODUK PRAKTIKUM/Little Star 15 menit Pintar Membaca/pintar_membaca-1.jpg", "price": "Rp39,000"},
            {"name": "Seri Amazing Science From Quran", "image": "FOTO PRODUK PRAKTIKUM/Seri Amazing Science From Quran/amazing_science-1.jpg", "price": "Rp48,000"},
            {"name": "Tanya Ini Tanya Itu Virus dan Bakteri", "image": "FOTO PRODUK PRAKTIKUM/Tanya Ini Tanya Itu Virus dan Bakteri/virus-1.jpg", "price": "Rp49,000"},
        ]
        
        row = 1
        column = 0
        
        for product in products:
            self.add_product(product, row, column)
            column += 1
            if column == 5:
                row += 1
                column = 0

        tk.Button(self.frame, text="Kembali ke Menu Utama", command=self.main_menu).grid(row=row+1, column=0, columnspan=5, pady=5)

    def add_product(self, product, row, column):
        frame = tk.Frame(self.frame)
        frame.grid(row=row, column=column, padx=5, pady=5)

        try:
            image_path = product["image"]
            image = Image.open(image_path)
            image = image.resize((100, 100))
            photo = ImageTk.PhotoImage(image)

            label = tk.Label(frame, image=photo)
            label.image = photo
            label.pack()

            name_label = tk.Label(frame, text=f"{product['price']}", font=('Arial', 10))
            name_label.pack()

            button = tk.Button(frame, text=f" {product['name']}", command=lambda: self.select_quantity(product))
            button.pack()
            
        except Exception as e:
            messagebox.showerror("Error", f"Error loading image for {product['name']}: {e}")
            return

    def select_quantity(self, product):
        self.clear_frame()
        self.frame = tk.Frame(self.root)
        self.frame.pack()

        tk.Label(self.frame, text=f"Pilih Jumlah untuk {product['name']}").pack(pady=10)
        tk.Label(self.frame, text="Masukkan jumlah produk:").pack()
        self.quantity_entry = tk.Entry(self.frame)
        self.quantity_entry.pack()
        tk.Button(self.frame, text="Tambah ke Keranjang", command=lambda: self.add_to_cart(product)).pack(pady=10)
        tk.Button(self.frame, text="Kembali ke Pemilihan Produk", command=self.select_product).pack(pady=5)

    def add_to_cart(self, product):
        quantity = self.quantity_entry.get()
        if quantity.isdigit() and int(quantity) > 0:
            self.cart.append({"product": product["name"], "quantity": int(quantity)})
            messagebox.showinfo("Keranjang", f"{quantity} {product['name']} ditambahkan ke keranjang!")
            self.checkout()
        else:
            messagebox.showerror("Keranjang", "Masukkan jumlah yang valid!")

    def view_cart(self):
        self.clear_frame()
        self.frame = tk.Frame(self.root)
        self.frame.pack()
        
        tk.Label(self.frame, text="Keranjang Belanja").pack(pady=10)
        
        if not self.cart:
            tk.Label(self.frame, text="Keranjang Anda kosong.").pack()
        else:
            total_price = 0
            for item in self.cart:
                product_name = item['product']
                quantity = item['quantity']
                price = self.get_product_price(product_name)
                total_price += price * quantity
                tk.Label(self.frame, text=f"Produk: {product_name} - Jumlah: {quantity} - Harga: Rp{price * quantity}").pack()
            
            tk.Label(self.frame, text=f"Total Harga: Rp{total_price}").pack()
        
        tk.Button(self.frame, text="Checkout", command=self.checkout).pack(pady=5)
        tk.Button(self.frame, text="Kembali ke Menu Utama", command=self.main_menu).pack(pady=5)

    def get_product_price(self, product_name):
        price_list = {
            "100 Kata Pertamaku": 89000,
            "Aktivitas Pertamaku ABC 123": 89000,
            "100 Hewan Pertamaku": 89000,
            "Aku Mau Tau Semua - Jilid 1": 59000,
            "Aku Mau Tau Semua - Jilid 2": 59000,
            "Eduscience Eksperimen Keren" : 64000,
            "Eduscience Hewan Predator" : 64000,
            "Flip & Find - Lets See Inside Animal!": 99000,
            "Kidzopedia Hewan Dan Tumbuhan": 89000,
            "Kidzopedia Keajaiban Tubuh Manusia": 89000,
            "Kidzopedia Peristiwa Alam": 89000,
            "Lets Explore! Around the Farm": 98000,
            "Little Star 15 menit Pintar Membaca": 39000,
            "Seri Amazing Science From Quran": 48000,
            "Tanya Ini Tanya Itu Virus dan Bakteri": 49000,
        }
        return price_list.get(product_name, 0) 
    
    def checkout(self):
        self.clear_frame()
        self.frame = tk.Frame(self.root)
        self.frame.pack()
        
        tk.Label(self.frame, text="Checkout").pack(pady=10)
        total_price = 0
        for item in self.cart:
            product_name = item['product']
            quantity = item['quantity']
            price = self.get_product_price(product_name)
            total_price += price * quantity
            tk.Label(self.frame, text=f"Produk: {product_name} - Jumlah: {quantity} - Harga: Rp{price}").pack()
            tk.Label(self.frame, text=f"Total Harga: Rp{total_price}").pack()
        
        tk.Button(self.frame, text="Lakukan Pembayaran", command=self.payment_method).pack(pady=10)
        tk.Button(self.frame, text="Pilih Produk Lainnya!!", command=self.select_product).pack(pady=5)
        tk.Button(self.frame, text="Kembali ke Menu Utama", command=self.main_menu).pack(pady=5)

    def payment_method(self):
        self.clear_frame()
        self.frame = tk.Frame(self.root)
        self.frame.pack()

        tk.Label(self.frame, text="Pilih Metode Pembayaran").pack(pady=10)

        payment_methods = ["BCA", "BRI", "BNI", "Mandiri","GoPay","OVO"]

        for i, method in enumerate(payment_methods):
            button = tk.Button(self.frame, text=method, command=lambda method=method: self.select_payment_method(method))
            button.pack(pady=5)

    def select_payment_method(self, method):
        self.transaction_data = {"products": self.cart, "payment": method}
        messagebox.showinfo("Pembayaran", "Pembayaran berhasil dikonfirmasi!")
        self.print_invoice()

    def confirm_payment(self):
        payment = self.payment_method.get()
        if payment:
            self.transaction_data = {"products": self.cart, "payment": payment}
            messagebox.showinfo("Pembayaran", "Pembayaran berhasil dikonfirmasi!")
            self.print_invoice()
        else:
            messagebox.showerror("Pembayaran", "Masukkan metode pembayaran!")

    def print_invoice(self):
        self.clear_frame()
        self.frame = tk.Frame(self.root)
        self.frame.pack()

        tk.Label(self.frame, text="Invoice").pack(pady=10)
        for item in self.transaction_data['products']:
            tk.Label(self.frame, text=f"Produk: {item['product']} - Jumlah: {item['quantity']}").pack()
        tk.Label(self.frame, text=f"Metode Pembayaran: {self.transaction_data['payment']}").pack()

        tk.Button(self.frame, text="Transaksi Baru", command=self.main_menu).pack(pady=10)
        tk.Button(self.frame, text="Cetak Invoice Dalam PDF", command=self.generate_pdf).pack(pady=10)
        tk.Button(self.frame, text="Kembali ke Menu Utama", command=self.main_menu).pack(pady=5)

        with open(self.user_details_file, 'r') as file:
            reader = csv.reader(file)
            next(reader) 
            for row in reader:
                if row[0] == self.transaction_data['products']:
                    full_name = row[0]
                    phone_number = row[1]
                    address = row[2]

        tk.Label(self.frame, text=f"Nama Lengkap: {full_name}").pack()
        tk.Label(self.frame, text=f"No Handphone: {phone_number}").pack()
        tk.Label(self.frame, text=f"Alamat: {address}").pack()


    def generate_pdf(self):
        file_path = fd.asksaveasfilename(defaultextension=".pdf", filetypes=[("PDF Files", "*.pdf")])

        if file_path:
            pdf = FPDF()
            pdf.add_page()
            pdf.set_font("Arial", size=12)

            pdf.cell(200, 10, txt="Invoice Pembelian", ln=True, align='C')
            pdf.ln(10)

            total_price = 0
            for item in self.transaction_data['products']:
                product_name = item['product']
                quantity = item['quantity']
                price = self.get_product_price(product_name)
                total_price += price * quantity
                pdf.cell(200, 10, txt=f"Produk: {product_name} - Jumlah: {quantity} - Harga: Rp{price * quantity}", ln=True)

            pdf.ln(10)
            pdf.cell(200, 10, txt=f"Total Harga: Rp{total_price}", ln=True)
            pdf.output(file_path)

    def clear_frame(self):
        for widget in self.root.winfo_children():
            widget.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = BookStoreApp(root)
    root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/tkinter/__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/np/6ymrwq053dl57g0rpsb7x2jm0000gn/T/ipykernel_3864/1453909643.py", line 469, in <lambda>
    button = tk.Button(self.frame, text=method, command=lambda method=method: self.select_payment_method(method))
                                                                              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/np/6ymrwq053dl57g0rpsb7x2jm0000gn/T/ipykernel_3864/1453909643.py", line 475, in select_payment_method
    self.print_invoice()
  File "/var/folders/np/6ymrwq053dl57g0rpsb7x2jm0000gn/T/ipykernel_3864/1453909643.py", line 509, in print_invoice
    tk.Label(self.frame, text=f"Nama Lengkap: {full_name}").pack()
                                               ^^^^^^^^^
UnboundLocalError: cannot access loc